In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

In [1]:
!pip install transformers[torch] datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")

Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [3]:
from datasets import concatenate_datasets
from datasets import DatasetDict

def contains_price(example):
    return ('price' in example['text'].lower() and 'high' in example['text'].lower()) or ('price' in example['text'].lower() and 'low' in example['text'].lower())

def contains_pricehighnegative(example):
    return 'price' in example['text'].lower() and 'high' in example['text'].lower() and example['label'] == 0

def contains_pricehighpositive(example):
    return 'price' in example['text'].lower() and 'high' in example['text'].lower() and example['label'] == 1

def contains_pricelownegative(example):
    return 'price' in example['text'].lower() and 'low' in example['text'].lower() and example['label'] == 0

def contains_pricelowpositive(example):
    return 'price' in example['text'].lower() and 'low' in example['text'].lower() and example['label'] == 1

filtered_dataset = dataset.filter(contains_price)

def change_label(example):
    # Change the label based on your condition
    if example['label'] > 2:
        example['label'] = 1
    else:
        example['label'] = 0
    return example

filtered_dataset = filtered_dataset.map(change_label)

pricehighnegative_train_unfilter = filtered_dataset.filter(contains_price)['train'].shuffle(seed=42).select([i for i in list(range(27000))])

pricehighnegative_train_balance = filtered_dataset.filter(contains_pricehighnegative)['train'].shuffle(seed=42).select([i for i in list(range(2900))])
pricehighpositive_train_balance = filtered_dataset.filter(contains_pricehighpositive)['train'].shuffle(seed=42).select([i for i in list(range(2900))])
pricelownegative_train_balance = filtered_dataset.filter(contains_pricelownegative)['train'].shuffle(seed=42).select([i for i in list(range(2900))])
pricelowpositive_train_balance = filtered_dataset.filter(contains_pricelowpositive)['train'].shuffle(seed=42).select([i for i in list(range(2900))])

pricehighnegative_validation_balance = filtered_dataset.filter(contains_pricehighnegative)['train'].shuffle(seed=42).select([i for i in list(range(2900,3100))])
pricehighpositive_validation_balance = filtered_dataset.filter(contains_pricehighpositive)['train'].shuffle(seed=42).select([i for i in list(range(2900,3100))])
pricelownegative_validation_balance = filtered_dataset.filter(contains_pricelownegative)['train'].shuffle(seed=42).select([i for i in list(range(2900,3100))])
pricelowpositive_validation_balance = filtered_dataset.filter(contains_pricelowpositive)['train'].shuffle(seed=42).select([i for i in list(range(2900,3100))])

pricehighnegative_train_imbalance = filtered_dataset.filter(contains_pricehighnegative)['train'].shuffle(seed=42).select([i for i in list(range(2900))])
pricehighpositive_train_imbalance = filtered_dataset.filter(contains_pricehighpositive)['train'].shuffle(seed=42).select([i for i in list(range(500))])
pricelownegative_train_imbalance = filtered_dataset.filter(contains_pricelownegative)['train'].shuffle(seed=42).select([i for i in list(range(500))])
pricelowpositive_train_imbalance = filtered_dataset.filter(contains_pricelowpositive)['train'].shuffle(seed=42).select([i for i in list(range(2900))])

newdataset_train_unfilter = pricehighnegative_train_unfilter
newdataset_train_balance = concatenate_datasets([pricehighnegative_train_balance, pricehighpositive_train_balance, pricelownegative_train_balance, pricelowpositive_train_balance])
newdataset_train_imbalance = concatenate_datasets([pricehighnegative_train_imbalance, pricehighpositive_train_imbalance, pricelownegative_train_imbalance, pricelowpositive_train_imbalance])
newdataset_validation = concatenate_datasets([pricehighnegative_validation_balance, pricehighpositive_validation_balance, pricelownegative_validation_balance, pricelowpositive_validation_balance])

# train_test_split_ratio = 0.1  # 10% for validation
# split_datasets_unfilter = newdataset_train_unfilter.train_test_split(test_size=train_test_split_ratio)
# split_datasets_balance = newdataset_train_balance.train_test_split(test_size=train_test_split_ratio)
# split_datasets_imbalance = newdataset_train_imbalance.train_test_split(test_size=train_test_split_ratio)

'''
Test tests for four groups
'''
pricehighnegative_test = filtered_dataset.filter(contains_pricehighnegative)['test'].shuffle(seed=42).select([i for i in list(range(230))]) #group 1
pricehighpositive_test = filtered_dataset.filter(contains_pricehighpositive)['test'].shuffle(seed=42).select([i for i in list(range(230))]) #group 2
pricelownegative_test = filtered_dataset.filter(contains_pricelownegative)['test'].shuffle(seed=42).select([i for i in list(range(230))]) #group 3
pricelowpositive_test = filtered_dataset.filter(contains_pricelowpositive)['test'].shuffle(seed=42).select([i for i in list(range(230))]) #group 4

'''
Final Dataset
'''
final_dataset_unfilter = DatasetDict({
    'train': newdataset_train_unfilter,
    'validation': newdataset_validation,
    'test_group_1': pricehighnegative_test,
    'test_group_2': pricehighpositive_test,
    'test_group_3': pricelownegative_test,
    'test_group_4': pricelowpositive_test,
})

final_dataset_balance = DatasetDict({
    'train': newdataset_train_balance,
    'validation': newdataset_validation,
    'test_group_1': pricehighnegative_test,
    'test_group_2': pricehighpositive_test,
    'test_group_3': pricelownegative_test,
    'test_group_4': pricelowpositive_test,
})

final_dataset_imbalance = DatasetDict({
    'train': newdataset_train_imbalance,
    'validation': newdataset_validation,
    'test_group_1': pricehighnegative_test,
    'test_group_2': pricehighpositive_test,
    'test_group_3': pricelownegative_test,
    'test_group_4': pricelowpositive_test,
})
print(final_dataset_unfilter)
print(final_dataset_balance)
print(final_dataset_imbalance)

Filter:   0%|          | 0/650000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/40779 [00:00<?, ? examples/s]

Map:   0%|          | 0/3089 [00:00<?, ? examples/s]

Filter:   0%|          | 0/40779 [00:00<?, ? examples/s]

...

Filter:   0%|          | 0/40779 [00:00<?, ? examples/s]

Filter:   0%|          | 0/3089 [00:00<?, ? examples/s]

Filter:   0%|          | 0/40779 [00:00<?, ? examples/s]

Filter:   0%|          | 0/3089 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 27000
    })
    validation: Dataset({
        features: ['label', 'text'],
        num_rows: 800
    })
    test_group_1: Dataset({
        features: ['label', 'text'],
        num_rows: 230
    })
    test_group_2: Dataset({
        features: ['label', 'text'],
        num_rows: 230
    })
    test_group_3: Dataset({
        features: ['label', 'text'],
        num_rows: 230
    })
    test_group_4: Dataset({
        features: ['label', 'text'],
        num_rows: 230
    })
})
DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 11600
    })
    validation: Dataset({
        features: ['label', 'text'],
        num_rows: 800
    })
    test_group_1: Dataset({
        features: ['label', 'text'],
        num_rows: 230
    })
    test_group_2: Dataset({
        features: ['label', 'text'],
        num_rows: 230
    })
    test_group_3: Dataset({
        feature

Pre-trained Roberta


In [ ]:
import torch
from transformers import RobertaConfig, RobertaModel, AutoTokenizer, RobertaForSequenceClassification, pipeline
from transformers import RobertaTokenizerFast

In [ ]:
#configuration = RobertaConfig()
tokenizer = RobertaTokenizerFast.from_pretrained("cardiffnlp/twitter-roberta-base-emotion")
#tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-emotion")
model = RobertaForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-emotion")

max_length = 512
classifier = pipeline("sentiment-analysis", model = model, tokenizer = tokenizer, truncation=True, padding=True,  max_length=max_length)
results_1 = classifier(pricehighnegative_test["text"], truncation=True, padding='max_length', max_length=max_length)
results_2 = classifier(pricehighpositive_test["text"], truncation=True, padding='max_length', max_length=max_length)
results_3 = classifier(pricelownegative_test["text"], truncation=True, padding='max_length', max_length=max_length)
results_4 = classifier(pricelowpositive_test["text"], truncation=True, padding='max_length', max_length=max_length)
for result in results_1:
    print(f"label: {result['label']}, with score: {round(result['score'], 4)}")

label: joy, with score: 0.5104
label: optimism, with score: 0.5634
label: joy, with score: 0.7508
label: joy, with score: 0.8931
label: optimism, with score: 0.869
label: joy, with score: 0.8518
label: joy, with score: 0.8341
label: optimism, with score: 0.3682
label: sadness, with score: 0.9686
label: optimism, with score: 0.9378
label: joy, with score: 0.7001
label: joy, with score: 0.9706
label: joy, with score: 0.9838
label: joy, with score: 0.6613
label: joy, with score: 0.9745
label: optimism, with score: 0.7613
label: joy, with score: 0.4652
label: joy, with score: 0.9451
label: optimism, with score: 0.7707
label: joy, with score: 0.9638
label: joy, with score: 0.9786
label: joy, with score: 0.4857
label: sadness, with score: 0.4727
label: anger, with score: 0.35
label: joy, with score: 0.9326
label: joy, with score: 0.9665
label: optimism, with score: 0.9159
label: joy, with score: 0.5334
label: joy, with score: 0.4878
label: optimism, with score: 0.5599
label: joy, with score:

In [ ]:
label_mapping = {
    "joy": 0,
    "anger": 1,
    "sadness": 2,
    "fear": 3,
    "neutral": 4,
    "optimism": 5
}

predicted_labels = [label_mapping[pred['label'].split('_')[-1]] for pred in results_1]
true_labels = pricehighnegative_test['label']
accuracy = sum([pred == true for pred, true in zip(predicted_labels, true_labels)]) / len(true_labels)
print(f"Test 1 Accuracy: {accuracy:.4f}")

predicted_labels = [label_mapping[pred['label'].split('_')[-1]] for pred in results_2]
true_labels = pricehighpositive_test['label']
accuracy = sum([pred == true for pred, true in zip(predicted_labels, true_labels)]) / len(true_labels)
print(f"Test 2 Accuracy: {accuracy:.4f}")

predicted_labels = [label_mapping[pred['label'].split('_')[-1]] for pred in results_3]
true_labels = pricelownegative_test['label']
accuracy = sum([pred == true for pred, true in zip(predicted_labels, true_labels)]) / len(true_labels)
print(f"Test 3 Accuracy: {accuracy:.4f}")

predicted_labels = [label_mapping[pred['label'].split('_')[-1]] for pred in results_4]
true_labels = pricelowpositive_test['label']
accuracy = sum([pred == true for pred, true in zip(predicted_labels, true_labels)]) / len(true_labels)
print(f"Test 4 Accuracy: {accuracy:.4f}")

Test 1 Accuracy: 0.6217
Test 2 Accuracy: 0.0087
Test 3 Accuracy: 0.6522
Test 4 Accuracy: 0.0043


In [7]:
!pip uninstall accelerate transformers

Found existing installation: accelerate 0.25.0
Uninstalling accelerate-0.25.0:
  Would remove:
    /usr/local/bin/accelerate
    /usr/local/bin/accelerate-config
    /usr/local/bin/accelerate-estimate-memory
    /usr/local/bin/accelerate-launch
    /usr/local/lib/python3.10/dist-packages/accelerate-0.25.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/accelerate/*
Proceed (Y/n)? y
  Successfully uninstalled accelerate-0.25.0
Found existing installation: transformers 4.36.0
Uninstalling transformers-4.36.0:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/transformers-4.36.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/transformers/*
Proceed (Y/n)? y
  Successfully uninstalled transformers-4.36.0


In [8]:
!pip install accelerate
!pip install evaluate
!pip install transformers[torch]

  Using cached accelerate-0.25.0-py3-none-any.whl (265 kB)


  Using cached transformers-4.36.0-py3-none-any.whl (8.2 MB)


unfilter data

In [9]:
from transformers import RobertaConfig, RobertaModel, AutoTokenizer, RobertaForSequenceClassification, pipeline

import torch
# tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-emotion")
model = RobertaForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-emotion")

from transformers import AutoTokenizer, DataCollatorWithPadding

from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")

from transformers import Trainer

from transformers import RobertaTokenizerFast
tokenizer = RobertaTokenizerFast.from_pretrained("cardiffnlp/twitter-roberta-base-emotion")

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
def tokenize_and_pad(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)
tokenized_datasets_unfilter = final_dataset_unfilter.map(tokenize_and_pad, batched=True)


# def tokenize_function(example):
#    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=512)

# tokenized_datasets = new_dataset.map(tokenize_function, batched=True)


trainer_unfilter = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets_unfilter["train"],
    eval_dataset=tokenized_datasets_unfilter["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer_unfilter.train()

predictions_test_group_1 = trainer_unfilter.predict(tokenized_datasets_unfilter["test_group_1"])
predictions_test_group_2 = trainer_unfilter.predict(tokenized_datasets_unfilter["test_group_2"])
predictions_test_group_3 = trainer_unfilter.predict(tokenized_datasets_unfilter["test_group_3"])
predictions_test_group_4 = trainer_unfilter.predict(tokenized_datasets_unfilter["test_group_4"])




import evaluate

import numpy as np

preds_test_group_1 = np.argmax(predictions_test_group_1.predictions, axis=-1)
preds_test_group_2 = np.argmax(predictions_test_group_2.predictions, axis=-1)
preds_test_group_3 = np.argmax(predictions_test_group_3.predictions, axis=-1)
preds_test_group_4 = np.argmax(predictions_test_group_4.predictions, axis=-1)

metric = evaluate.load("glue", "mrpc")
#metric.compute(predictions=preds, references=predictions.label_ids)


Map:   0%|          | 0/27000 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/230 [00:00<?, ? examples/s]

Map:   0%|          | 0/230 [00:00<?, ? examples/s]

Map:   0%|          | 0/230 [00:00<?, ? examples/s]

Map:   0%|          | 0/230 [00:00<?, ? examples/s]

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,0.446900
1000,0.417900
1500,0.362300
2000,0.375200
2500,0.340500
3000,0.343400
3500,0.319300
4000,0.265900
4500,0.257500
5000,0.296900


In [10]:
metric = evaluate.load("glue", "mrpc")
print("group 1")
metric.compute(predictions=preds_test_group_1, references=predictions_test_group_1.label_ids)

group 1


{'accuracy': 0.9391304347826087, 'f1': 0.0}

In [11]:


print("group 2")
metric.compute(predictions=preds_test_group_2, references=predictions_test_group_2.label_ids)





group 2


{'accuracy': 0.808695652173913, 'f1': 0.8942307692307693}

In [12]:
print("group 3")
metric.compute(predictions=preds_test_group_3, references=predictions_test_group_3.label_ids)

group 3


{'accuracy': 0.9304347826086956, 'f1': 0.0}

In [13]:
print("group 4")
metric.compute(predictions=preds_test_group_4, references=predictions_test_group_4.label_ids)



group 4


{'accuracy': 0.8565217391304348, 'f1': 0.9227166276346604}

Balance


In [14]:
from transformers import RobertaConfig, RobertaModel, AutoTokenizer, RobertaForSequenceClassification, pipeline

import torch
# tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-emotion")
model = RobertaForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-emotion")

from transformers import AutoTokenizer, DataCollatorWithPadding

from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")

from transformers import Trainer

from transformers import RobertaTokenizerFast
tokenizer = RobertaTokenizerFast.from_pretrained("cardiffnlp/twitter-roberta-base-emotion")

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
def tokenize_and_pad(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)
tokenized_datasets_balance = final_dataset_balance.map(tokenize_and_pad, batched=True)


# def tokenize_function(example):
#    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=512)

# tokenized_datasets = new_dataset.map(tokenize_function, batched=True)


trainer_balance = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets_balance["train"],
    eval_dataset=tokenized_datasets_balance["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer_balance.train()

predictions_test_group_1 = trainer_balance.predict(tokenized_datasets_balance["test_group_1"])
predictions_test_group_2 = trainer_balance.predict(tokenized_datasets_balance["test_group_2"])
predictions_test_group_3 = trainer_balance.predict(tokenized_datasets_balance["test_group_3"])
predictions_test_group_4 = trainer_balance.predict(tokenized_datasets_balance["test_group_4"])




import evaluate

import numpy as np

preds_test_group_1 = np.argmax(predictions_test_group_1.predictions, axis=-1)
preds_test_group_2 = np.argmax(predictions_test_group_2.predictions, axis=-1)
preds_test_group_3 = np.argmax(predictions_test_group_3.predictions, axis=-1)
preds_test_group_4 = np.argmax(predictions_test_group_4.predictions, axis=-1)

metric = evaluate.load("glue", "mrpc")
#metric.compute(predictions=preds, references=predictions.label_ids)


Map:   0%|          | 0/11600 [00:00<?, ? examples/s]

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,0.477000
1000,0.375500
1500,0.359000
2000,0.269000
2500,0.258900
3000,0.233900
3500,0.117900
4000,0.138300


Checkpoint destination directory test-trainer/checkpoint-500 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory test-trainer/checkpoint-1000 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory test-trainer/checkpoint-1500 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory test-trainer/checkpoint-2000 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory test-trainer/checkpoint-2500 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory test-trainer/checkpoint-3000 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory test-trainer/checkpoint-3500 already exists and is non-empty.Saving will proceed

In [15]:
metric = evaluate.load("glue", "mrpc")
print("group 1")
metric.compute(predictions=preds_test_group_1, references=predictions_test_group_1.label_ids)




group 1


{'accuracy': 0.9304347826086956, 'f1': 0.0}

In [16]:

print("group 2")
metric.compute(predictions=preds_test_group_2, references=predictions_test_group_2.label_ids)

group 2


{'accuracy': 0.9, 'f1': 0.9473684210526316}

In [17]:

print("group 3")
metric.compute(predictions=preds_test_group_3, references=predictions_test_group_3.label_ids)

group 3


{'accuracy': 0.8565217391304348, 'f1': 0.0}

In [18]:

print("group 4")
metric.compute(predictions=preds_test_group_4, references=predictions_test_group_4.label_ids)



group 4


{'accuracy': 0.9217391304347826, 'f1': 0.9592760180995475}

In [19]:
from transformers import RobertaConfig, RobertaModel, AutoTokenizer, RobertaForSequenceClassification, pipeline

import torch
# tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-emotion")
model = RobertaForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-emotion")

from transformers import AutoTokenizer, DataCollatorWithPadding

from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")

from transformers import Trainer

from transformers import RobertaTokenizerFast
tokenizer = RobertaTokenizerFast.from_pretrained("cardiffnlp/twitter-roberta-base-emotion")

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
def tokenize_and_pad(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)
tokenized_datasets_imbalance = final_dataset_imbalance.map(tokenize_and_pad, batched=True)


# def tokenize_function(example):
#    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=512)

# tokenized_datasets = new_dataset.map(tokenize_function, batched=True)


trainer_imbalance = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets_imbalance["train"],
    eval_dataset=tokenized_datasets_imbalance["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

trainer_imbalance.train()

predictions_test_group_1 = trainer_unfilter.predict(tokenized_datasets_imbalance["test_group_1"])
predictions_test_group_2 = trainer_unfilter.predict(tokenized_datasets_imbalance["test_group_2"])
predictions_test_group_3 = trainer_unfilter.predict(tokenized_datasets_imbalance["test_group_3"])
predictions_test_group_4 = trainer_unfilter.predict(tokenized_datasets_imbalance["test_group_4"])




import evaluate

import numpy as np

preds_test_group_1 = np.argmax(predictions_test_group_1.predictions, axis=-1)
preds_test_group_2 = np.argmax(predictions_test_group_2.predictions, axis=-1)
preds_test_group_3 = np.argmax(predictions_test_group_3.predictions, axis=-1)
preds_test_group_4 = np.argmax(predictions_test_group_4.predictions, axis=-1)

metric = evaluate.load("glue", "mrpc")
#metric.compute(predictions=preds, references=predictions.label_ids)


Map:   0%|          | 0/6800 [00:00<?, ? examples/s]

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,0.456900
1000,0.323400
1500,0.234300
2000,0.175300
2500,0.110000


Checkpoint destination directory test-trainer/checkpoint-500 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory test-trainer/checkpoint-1000 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory test-trainer/checkpoint-1500 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory test-trainer/checkpoint-2000 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory test-trainer/checkpoint-2500 already exists and is non-empty.Saving will proceed but saved results may be invalid.


In [20]:
metric = evaluate.load("glue", "mrpc")
print("group 1")
metric.compute(predictions=preds_test_group_1, references=predictions_test_group_1.label_ids)




group 1


{'accuracy': 0.9391304347826087, 'f1': 0.0}

In [21]:
print("group 2")
metric.compute(predictions=preds_test_group_2, references=predictions_test_group_2.label_ids)

group 2


{'accuracy': 0.808695652173913, 'f1': 0.8942307692307693}

In [22]:

print("group 3")
metric.compute(predictions=preds_test_group_3, references=predictions_test_group_3.label_ids)


group 3


{'accuracy': 0.9304347826086956, 'f1': 0.0}

In [23]:


print("group 4")
metric.compute(predictions=preds_test_group_4, references=predictions_test_group_4.label_ids)


group 4


{'accuracy': 0.8565217391304348, 'f1': 0.9227166276346604}